In [1]:
!pip install --no-cache-dir -q transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 244.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 361.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 272.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 286.6 MB/s eta 0:00:00


In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.cuda() if torch.cuda.is_available() else None
model.eval();

def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [5]:
%%time
text = """Добрый день, техническая поддержка СПбГУ, слушаю вас.
Добрый день. Я забыла, к сожалению, свой пароль от почты, что мне нужно сделать?
Скажите пожалуйста свои фамилию, имя, отчество?
Филатова Мария Евгеньевна.
Хорошо, какой у вас факультет?
Матмех.
Какой у вас номер группы?
20 б 07 мм.
Ага, хорошо, с завтрашнего дня можете зайти в деканат, показать студенческий билет, и вам выдадут новый пароль.
А дистанционно никак не получится это сделать?
Простите, но мы не можем без удостоверения личности выдать вам новый пароль.
Ладно, хорошо, спасибо, до свидания.
До свидания."""
print(summarize(text, n_words=15))

Скажите, пожалуйста, свой пароль от почты.
CPU times: user 332 ms, sys: 0 ns, total: 332 ms
Wall time: 331 ms


In [6]:
%%time
text = """Высота башни составляет 324 метра (1063 фута), примерно такая же высота, как у 81-этажного здания, и самое высокое сооружение в Париже. Его основание квадратно, размером 125 метров (410 футов) с любой стороны. Во время строительства Эйфелева башня превзошла монумент Вашингтона, став самым высоким искусственным сооружением в мире, и этот титул она удерживала в течение 41 года до завершения строительство здания Крайслер в Нью-Йорке в 1930 году. Это первое сооружение которое достигло высоты 300 метров. Из-за добавления вещательной антенны на вершине башни в 1957 году она сейчас выше здания Крайслер на 5,2 метра (17 футов). За исключением передатчиков, Эйфелева башня является второй самой высокой отдельно стоящей структурой во Франции после виадука Мийо."""
print(summarize(text, n_words=15))

Французская Эйфелева башня достигла высоты 300 метров.
CPU times: user 621 ms, sys: 0 ns, total: 621 ms
Wall time: 623 ms
